In [66]:
import pandas as pd

import os
from glob import glob
import pickle
from sklearn.metrics import f1_score, balanced_accuracy_score, precision_score, recall_score, confusion_matrix

In [67]:
def get_results(path, dataset, seed):

    dataset_path = f'../data/{dataset}.csv' if dataset=='expert' else f'../data/{dataset}_{seed}.csv'
    df = pd.read_csv(dataset_path)
    
    with open(path, 'rb') as f:
        res_vals = pickle.load(f)
        
    df.loc[res_vals['idx'], 'labels'] = res_vals['labels']
    df.loc[res_vals['idx'], 'predictions'] = res_vals['predictions']
    df.loc[res_vals['idx'], 'neg_prob'] = res_vals['probabilities'][:,0]
    df.loc[res_vals['idx'], 'pos_prob'] = res_vals['probabilities'][:,1]
    
    if dataset=='expert':
        df['labels'] = (df['labels'].astype(bool) & ~df['Konsensus Target 1'].isna()).astype(float)

    return df

In [68]:
def calculate_metrics(predictions, labels):
    
    res = []

    res.append(balanced_accuracy_score(predictions, labels))
    res.append(f1_score(predictions, labels, average='binary'))
    res.append(precision_score(predictions, labels, average='binary'))
    res.append(recall_score(predictions, labels, average='binary'))
    res.append(f1_score(1 - predictions, 1 - labels, average='binary'))

    return res

In [69]:
directory = '../data/outputs'

In [70]:
df_results = pd.DataFrame(columns=['model_name', 'optimizer', 'class_weights', 'batch_size', 'seed', 'dataset', 'balanced_accuracy', 'f1', 'precision', 'recall', 'f1_neg'])
for subdir, dirs, files in os.walk(directory):
    for file in files:
        try:
            info = subdir.split('/')
            model_name = info[3]
            optimizer = info[4]
            class_weights = info[5][-1] == '1'
            batch_size = int(info[6][-2:])
            seed = int(info[7][-2:])
            dataset = file[:-12]
            df = get_results(os.path.join(subdir, file), dataset, seed)
            params = [model_name, optimizer, class_weights, batch_size, seed, dataset]
            metrics = calculate_metrics(df['predictions'], df['labels'])
            df_results.loc[len(df_results)] = params + metrics
        except:
            pass

/home/rasteiger/miniconda3/envs/dslab/lib/python3.8/site-packages/sklearn/metrics/_classification.py:2394: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/rasteiger/miniconda3/envs/dslab/lib/python3.8/site-packages/sklearn/metrics/_classification.py:2394: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


In [71]:
df_results.sort_values(by='f1', ascending=False)

,model_name,optimizer,class_weights,batch_size,seed,dataset,balanced_accuracy,f1,precision,recall,f1_neg
3,e5,adamw,True,16,42,expert,0.760465,0.677419,0.709459,0.648148,0.855072
1,e5,adamw,True,16,43,expert,0.760000,0.664430,0.668919,0.660000,0.857550
51,e5_lora_nli_all,adamw,True,16,43,expert,0.752870,0.655518,0.662162,0.649007,0.853067
53,e5_lora_nli_all,adamw,True,16,44,expert,0.750476,0.651007,0.655405,0.646667,0.851852
5,e5,adamw,True,16,44,expert,0.750484,0.646259,0.641892,0.650685,0.852691
...,...,...,...,...,...,...,...,...,...,...,...
64,e5no_train,adamw,True,16,44,test,0.178134,0.302400,1.000000,0.178134,0.000000
63,e5no_train,adamw,True,16,43,expert,0.517904,0.273438,0.236486,0.324074,0.750000
61,e5no_train,adamw,True,16,42,expert,0.552068,0.207921,0.141892,0.388889,0.799499
60,e5no_train,adamw,True,16,42,test,0.527567,0.197960,0.176360,0.225590,0.848709


In [72]:
df = df_results.groupby(['model_name', 'optimizer', 'class_weights', 'batch_size', 'dataset']).agg(['mean', 'std']).drop('seed', axis=1).sort_values(by=('f1', 'mean'), ascending=False)
df.to_csv('../data/experiment_results.csv')
df

balanced_accuracy  \
                                                                        mean   
model_name      optimizer class_weights batch_size dataset                     
e5              adamw     True          16         expert           0.756983   
e5_lora_nli_all adamw     True          16         expert           0.749664   
e5_lora_nli     adamw     True          16         expert           0.749213   
e5_nli_all      adamw     True          16         test             0.732539   
e5_nli          adamw     True          16         expert           0.733405   
                                                   test             0.732655   
e5_nli_all      adamw     True          16         expert           0.728035   
e5              adamw     True          16         test             0.726756   
roberta         adamw     True          16         expert           0.734335   
                                                   test             0.723662   
e5_lora_nli     adamw     True          16         test             0.722646   
e5_lora         adamw     True          16         expert           0.735082   
e5_lora_nli_all adamw     True          16         test             0.721535   
e5_lora         adamw     True          16         test             0.719894   
roberta_lora    adamw     True          16         test             0.715759   
                                                   expert           0.732715   
bert            adamw     True          16         test             0.710856   
bert_lora       adamw     True          16         test             0.704566   
bert            adamw     True          16         expert           0.700776   
bert_lora       adamw     True          16         expert           0.679901   
e5no_train      adamw     True          16         expert           0.455324   
                                                   test             0.400127   

                                                                      \
                                                                 std   
model_name      optimizer class_weights batch_size dataset             
e5              adamw     True          16         expert   0.005633   
e5_lora_nli_all adamw     True          16         expert   0.003680   
e5_lora_nli     adamw     True          16         expert   0.007365   
e5_nli_all      adamw     True          16         test     0.000528   
e5_nli          adamw     True          16         expert   0.013359   
                                                   test     0.003049   
e5_nli_all      adamw     True          16         expert   0.004125   
e5              adamw     True          16         test     0.003395   
roberta         adamw     True          16         expert   0.007790   
                                                   test     0.002738   
e5_lora_nli     adamw     True          16         test     0.002969   
e5_lora         adamw     True          16         expert   0.013629   
e5_lora_nli_all adamw     True          16         test     0.001058   
e5_lora         adamw     True          16         test     0.003162   
roberta_lora    adamw     True          16         test     0.002737   
                                                   expert   0.014986   
bert            adamw     True          16         test     0.002028   
bert_lora       adamw     True          16         test     0.003649   
bert            adamw     True          16         expert   0.010748   
bert_lora       adamw     True          16         expert   0.027005   
e5no_train      adamw     True          16         expert   0.139032   
                                                   test     0.192953   

                                                                  f1  \
                                                                mean   
model_name      optimizer class_weights batch_size dataset             
e5              adamw     True          1

In [74]:
df_results.groupby(['model_name', 'optimizer', 'class_weights', 'batch_size', 'dataset']).agg(['count'])

seed  \
                                                           count   
model_name      optimizer class_weights batch_size dataset         
bert            adamw     True          16         expert      3   
                                                   test        3   
bert_lora       adamw     True          16         expert      3   
                                                   test        3   
e5              adamw     True          16         expert      3   
                                                   test        3   
e5_lora         adamw     True          16         expert      3   
                                                   test        3   
e5_lora_nli     adamw     True          16         expert      3   
                                                   test        3   
e5_lora_nli_all adamw     True          16         expert      3   
                                                   test        3   
e5_nli          adamw     True          16         expert      3   
                                                   test        3   
e5_nli_all      adamw     True          16         expert      3   
                                                   test        3   
e5no_train      adamw     True          16         expert      3   
                                                   test        3   
roberta         adamw     True          16         expert      3   
                                                   test        3   
roberta_lora    adamw     True          16         expert      3   
                                                   test        3   

                                                           balanced_accuracy  \
                                                                       count   
model_name      optimizer class_weights batch_size dataset                     
bert            adamw     True          16         expert                  3   
                                                   test                    3   
bert_lora       adamw     True          16         expert                  3   
                                                   test                    3   
e5              adamw     True          16         expert                  3   
                                                   test                    3   
e5_lora         adamw     True          16         expert                  3   
                                                   test                    3   
e5_lora_nli     adamw     True          16         expert                  3   
                                                   test                    3   
e5_lora_nli_all adamw     True          16         expert                  3   
                                                   test                    3   
e5_nli          adamw     True          16         expert                  3   
                                                   test                    3   
e5_nli_all      adamw     True          16         expert                  3   
                                                   test                    3   
e5no_train      adamw     True          16         expert                  3   
                                                   test                    3   
roberta         adamw     True          16         expert                  3   
                                                   test                    3   
roberta_lora    adamw     True          16         expert                  3   
                                                   test                    3   

                                                              f1 precision  \
                                                           count     count   
model_name      optimizer class_weights batch_size dataset                   
bert            adamw     True          16         expert      3         3   
                                              